In [105]:
import pandas as pd
import numpy as np
import datetime
#import pandas_profiling # pip install pandas_profiling

taxiCancellationScore = pd.read_csv('Kaggle_YourCabs_score.csv')
taxiCancellationTraining = pd.read_csv('Kaggle_YourCabs_training.csv')

In [106]:
#taxiCancellationScore.Profile_Report()

In [107]:
taxiCancellationTraining.columns

Index(['id', 'user_id', 'vehicle_model_id', 'package_id', 'travel_type_id',
       'from_area_id', 'to_area_id', 'from_city_id', 'to_city_id', 'from_date',
       'to_date', 'online_booking', 'mobile_site_booking', 'booking_created',
       'from_lat', 'from_long', 'to_lat', 'to_long', 'Car_Cancellation',
       'Cost_of_error'],
      dtype='object')

In [108]:
taxiCancellationTraining.isnull().sum()

id                         0
user_id                    0
vehicle_model_id           0
package_id             35881
travel_type_id             0
from_area_id              88
to_area_id              9138
from_city_id           27086
to_city_id             41843
from_date                  0
to_date                17890
online_booking             0
mobile_site_booking        0
booking_created            0
from_lat                  93
from_long                 93
to_lat                  9138
to_long                 9138
Car_Cancellation           0
Cost_of_error              0
dtype: int64

### Drop 9138 rows, Drop from_city_id, Drop row from_lat

In [109]:
taxiCancellationScore.isnull().sum()

id                         0
user_id                    0
vehicle_model_id           0
package_id              8215
travel_type_id             0
from_area_id              20
to_area_id              2150
from_city_id            6288
to_city_id              9635
from_date                  0
to_date                 4127
online_booking             0
mobile_site_booking        0
booking_created            0
from_lat                  20
from_long                 20
to_lat                  2150
to_long                 2150
Unnamed: 18            10000
Unnamed: 19            10000
dtype: int64

#Convert Date String into Timestamp()
def date_time(y,x):
    from_date= [None]*y.shape[0]
    for i in range(y.shape[0]):
        if (type(y[x][i] is str and y[x][i].isnull() == False)):
            try:
                datetime_ob = datetime.datetime.strptime(y[x][i],"%m/%d/%Y %H:%M").weekday()
            except:
                print(int)
            y.loc[:,x].replace(to_replace=y.loc[:,x].values[i], value = datetime_ob, inplace = True)

In [110]:
#convert try 2
from_weekdate_test = [] #days of week
from_time_test = []
for x in taxiCancellationScore['from_date']:
    if type(x) is str:
        ziz = x.split(' ')
        day = ziz[0].split('/')
        for z in range(len(day)):
            day[z] = int(day[z])
        from_weekdate_test.append(datetime.date(day[2], day[0], day[1]).weekday())
        #from_weekdate is now ints as day of week, 0=sunday etc

        time = ziz[1].split(':')
        from_time_test.append(int(time[0])*60 + int(time[1]))
        #from_time is now ints as minute of day
        
        
booking_weekdate_test = [] #days of week
booking_time_test = []
for x in taxiCancellationScore['booking_created']:
    if type(x) is str:
        ziz = x.split(' ')
        day = ziz[0].split('/')
        for z in range(len(day)):
            day[z] = int(day[z])
        booking_weekdate_test.append(datetime.date(day[2], day[0], day[1]).weekday())
        #booking_weekdate is now ints as day of week, 0=sunday etc

        time = ziz[1].split(':')
        booking_time_test.append(int(time[0])*60 + int(time[1]))
        #booking_time is now ints as minute of day
        
#to be appended: from_weekdate, from_time, booking_weekdate, booking_time

In [111]:
#convert try 2
from_weekdate = [] #days of week
from_time = []
for x in taxiCancellationTraining['from_date']:
    if type(x) is str:
        ziz = x.split(' ')
        day = ziz[0].split('/')
        for z in range(len(day)):
            day[z] = int(day[z])
        from_weekdate.append(datetime.date(day[2], day[0], day[1]).weekday())
        #from_weekdate is now ints as day of week, 0=sunday etc

        time = ziz[1].split(':')
        from_time.append(int(time[0])*60 + int(time[1]))
        #from_time is now ints as minute of day
        
        
booking_weekdate = [] #days of week
booking_time = []
for x in taxiCancellationTraining['booking_created']:
    if type(x) is str:
        ziz = x.split(' ')
        day = ziz[0].split('/')
        for z in range(len(day)):
            day[z] = int(day[z])
        booking_weekdate.append(datetime.date(day[2], day[0], day[1]).weekday())
        #booking_weekdate is now ints as day of week, 0=sunday etc

        time = ziz[1].split(':')
        booking_time.append(int(time[0])*60 + int(time[1]))
        #booking_time is now ints as minute of day
        
#to be appended: from_weekdate, from_time, booking_weekdate, booking_time

In [112]:
date_df = pd.DataFrame({'from_weekdate':from_weekdate,'from_time':from_time,'booking_weekdate':booking_weekdate,'booking_time':booking_time})
new_taxiCancellationTraining = taxiCancellationTraining.join(date_df)

#Replace Trained Value in DataFrame
date_time(taxiCancellationScore,'from_date')
taxiCancellationScore.loc[:,'from_date']

date_time(taxiCancellationScore,'booking_created')
taxiCancellationScore.loc[:,'booking_created']

## Addressing Null values

In [113]:
new_taxiCancellationTraining.dropna(subset=['to_area_id', 'to_lat', 'to_long', ], inplace = True)

In [114]:
new_taxiCancellationTraining.isnull().sum()

id                         0
user_id                    0
vehicle_model_id           0
package_id             34293
travel_type_id             0
from_area_id               0
to_area_id                 0
from_city_id           19788
to_city_id             34293
from_date                  0
to_date                15186
online_booking             0
mobile_site_booking        0
booking_created            0
from_lat                   0
from_long                  0
to_lat                     0
to_long                    0
Car_Cancellation           0
Cost_of_error              0
from_weekdate              0
from_time                  0
booking_weekdate           0
booking_time               0
dtype: int64

In [115]:
new_taxiCancellationTraining

,id,user_id,vehicle_model_id,package_id,travel_type_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,...,from_lat,from_long,to_lat,to_long,Car_Cancellation,Cost_of_error,from_weekdate,from_time,booking_weekdate,booking_time
0,132512,22177,28,NaN,2,83.0,448.0,NaN,NaN,1/1/2013 2:00,...,12.924150,77.672290,12.927320,77.635750,0,1.0,1,120,1,99
1,132513,21413,12,NaN,2,1010.0,540.0,NaN,NaN,1/1/2013 9:00,...,12.966910,77.749350,12.927680,77.626640,0,1.0,1,540,1,145
2,132514,22178,12,NaN,2,1301.0,1034.0,NaN,NaN,1/1/2013 3:30,...,12.937222,77.626915,13.047926,77.597766,0,1.0,1,210,1,188
3,132515,13034,12,NaN,2,768.0,398.0,NaN,NaN,1/1/2013 5:45,...,12.989990,77.553320,12.971430,77.639140,0,1.0,1,345,1,279
4,132517,22180,12,NaN,2,1365.0,849.0,NaN,NaN,1/1/2013 9:00,...,12.845653,77.677925,12.954340,77.600720,0,1.0,1,540,1,473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43425,185936,30220,12,NaN,2,585.0,1226.0,15.0,NaN,11/25/2013 0:00,...,12.976770,77.572700,13.015080,77.677960,0,1.0,0,0,6,864
43426,185937,14364,12,NaN,2,1147.0,452.0,15.0,NaN,11/24/2013 18:00,...,13.030640,77.649100,12.952780,77.590880,0,1.0,6,1080,6,865
43427,185938,48727,12,NaN,2,393.0,466.0,15.0,NaN,11/26/2013 1:00,...,13.199560,77.706880,13.017436,77.644580,0,1.0,1,60,6,870
43429,185940,30724,87,NaN,2,1263.0,542.0,15.0,NaN,11/24/2013 16:00,...,12.968970,77.594560,12.938230,77.622890,0,1.0,6,960,6,883


In [116]:
feature_cols = ['vehicle_model_id', 'travel_type_id',
                'from_area_id', 'to_area_id', 
       'online_booking', 'mobile_site_booking',
       'from_lat', 'from_long', 'to_lat', 'to_long', 'from_weekdate','from_time','booking_weekdate','booking_time']

In [117]:
X = new_taxiCancellationTraining.loc[:,feature_cols] # Features
y = new_taxiCancellationTraining.loc[:,'Car_Cancellation'] # Target variable
X.isnull().sum()

vehicle_model_id       0
travel_type_id         0
from_area_id           0
to_area_id             0
online_booking         0
mobile_site_booking    0
from_lat               0
from_long              0
to_lat                 0
to_long                0
from_weekdate          0
from_time              0
booking_weekdate       0
booking_time           0
dtype: int64

In [118]:
X.shape
#34k rows

(34293, 14)

In [119]:
y.isnull().sum()

0

## Not append the Test dataframe yet

In [120]:
date_df_test = pd.DataFrame({'from_weekdate':from_weekdate_test,'from_time':from_time_test,'booking_weekdate':booking_weekdate_test,'booking_time':booking_time_test})
new_taxiCancellationScore =taxiCancellationScore.join(date_df_test)

In [121]:
new_taxiCancellationScore.dropna(subset=['to_area_id', 'to_lat', 'to_long', ], inplace = True)
X_test = new_taxiCancellationScore.loc[:,feature_cols] # Features
X_test.isnull().sum()

vehicle_model_id       0
travel_type_id         0
from_area_id           0
to_area_id             0
online_booking         0
mobile_site_booking    0
from_lat               0
from_long              0
to_lat                 0
to_long                0
from_weekdate          0
from_time              0
booking_weekdate       0
booking_time           0
dtype: int64

## 4 Models

In [122]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

#Logistic Regresstion

In [123]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
logreg.fit(X, y)

c:\users\prathna\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [124]:
#Deep Learning Keras Model
#relu -linear
#sigmoid function for final layer
#Dropout 50%
#Dimension input features column
#Output binary classification 0 and 1
model = Sequential()
model.add(Dense(124, activation='relu', input_dim = X.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()
#Using binary_crossentropy
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 124)               1860      
_________________________________________________________________
dropout_7 (Dropout)          (None, 124)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8000      
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                

In [125]:
#KNN
n_neigh = 3
from sklearn.neighbors import KNeighborsClassifier
neighbor = KNeighborsClassifier(n_neigh)
neighbor.fit(X, y) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [126]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
gaus = GaussianNB()
gaus.fit(X, y)

GaussianNB(priors=None, var_smoothing=1e-09)

# Prediction

In [127]:
model.fit(X,y, epochs=10, batch_size = 2, verbose = 1)

Epoch 1/10
10176/34293 [=======>......................] - ETA: 49s - loss: 2.1058 - accuracy: 0.8874

KeyboardInterrupt: 

In [ ]:
model.save('trained_model.h5')

In [ ]:
X_test.isnull().sum()

In [ ]:
y_pred_logreg = logreg.predict(X_test)
y_pred_keras= model.predict(X_test)
y_pred_neigh= neighbor.predict(X_test)
y_pred_gaus= gaus.predict(X_test)

# Performance of Each Model

# visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from sklearn import metrics
import numpy as np

cnf_matrix = metrics.confusion_matrix(y, y_pred_logreg)
cnf_matrix

In [ ]:
class_names=['Not play','Play'] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
# create heatmap
sns.heatmap(cnf_matrix, annot=True, cmap="YlGnBu" ,fmt='d', xticklabels=class_names, yticklabels=class_names)
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

